In [60]:
import pandas as pd
import glob
import os
import shutil
import re
from joblib import Parallel, delayed
import nibabel as nib
import numpy as np
import random
import seaborn as sns
from datetime import datetime

In [15]:
import sys
sys.path.append('/media/raid/orco/data/ICH/code/neurorad_bids_utils')
from bids_utils.utils import utils, mask_utils

In [11]:
preds = glob.glob('/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Dataset001_test/test*/OrbitAI_*.nii.gz')

In [32]:
df = pd.DataFrame(preds)
df.columns = ['prediction']
df['gt'] = df.prediction.apply(lambda x: '/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Dataset001_test/labelsTs/' + os.path.basename(x))
df['geom'] = df.prediction.apply(lambda x: '3d' if 'test_3d' in x else '2d' if 'test_2d' in x else 'NA')

In [33]:
df.head()

,prediction,gt,geom
0,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,3d
1,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,3d
2,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,3d
3,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,3d
4,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,3d


In [34]:
df.prediction[0]

'/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Dataset001_test/test_3d/OrbitAI_12321247.nii.gz'

In [53]:
res = []
for i,j in zip(list(df.prediction),list(df['gt'])):
    res.append([i,j] + utils.mp_dice_n(i, j, [1,2,3,4]) + [mask_utils.has_label(j,3), mask_utils.has_label(j,4), mask_utils.has_label(j,3), mask_utils.has_label(j,4)])

In [54]:
df = pd.DataFrame(res)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.948850,0.818869,0.000000,0.525356,False,True,False,True
1,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.868894,0.871292,0.000253,0.456496,True,True,True,True
2,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.892489,0.856616,0.000000,0.520145,False,True,False,True
3,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.901672,0.847869,0.001034,0.683816,True,True,True,True
4,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.943575,0.772958,0.108936,0.000000,True,False,True,False


In [55]:
df.columns = ['prediction', 'gt', 'DSC_eye', 'DSC_nerve', 'DSC_ret_det', 'DSC_tumor', 'has_ret_det', 'has_tumor', 'pred_ret_det', 'pred_tumor']
df['geom'] = df.prediction.apply(lambda x: '3d' if 'test_3d' in x else '2d' if 'test_2d' in x else 'NA')

In [56]:
df.head()

,prediction,gt,DSC_eye,DSC_nerve,DSC_ret_det,DSC_tumor,has_ret_det,has_tumor,pred_ret_det,pred_tumor,geom
0,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.948850,0.818869,0.000000,0.525356,False,True,False,True,3d
1,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.868894,0.871292,0.000253,0.456496,True,True,True,True,3d
2,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.892489,0.856616,0.000000,0.520145,False,True,False,True,3d
3,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.901672,0.847869,0.001034,0.683816,True,True,True,True,3d
4,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,/home/orco/data/Orbita_AI/nnUNet/nnUNet_raw/Da...,0.943575,0.772958,0.108936,0.000000,True,False,True,False,3d


In [57]:
df['sid'] = df.prediction.apply(lambda x: os.path.basename(x).split('_')[1].replace('.nii.gz',''))

In [62]:
date = '{}{}{}'.format(datetime.now().year, datetime.now().month, datetime.now().day)
date

'2023823'

In [64]:
df.to_csv('../data/test_dsc_{}.csv'.format(date))